In [ ]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

In [ ]:
pip install xgboost


In [ ]:
pip install --upgrade xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score,classification_report
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [ ]:
# Drop missing and duplicate values
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df = df[~(df['clean_comment'].str.strip() == '')]

# Import necessary libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r"[^A-Za-z0-9\s!?.]", '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

# Apply the preprocessing function to the `clean_comment` column
df['clean_comment'] = df['clean_comment'].apply(preprocess_comment)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
cleaned_dataset=df.dropna()

In [ ]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
cleaned_dataset['category'] = cleaned_dataset['category'].map({-1: 2, 0: 0, 1: 1})

In [ ]:
X_cleaned=cleaned_dataset['clean_comment']
y_cleaned=cleaned_dataset['category']

In [ ]:
# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

In [ ]:
X_train_tokenized=[comment.split() for comment in X_train_cleaned]
X_test_tokenized=[comment.split() for comment in X_test_cleaned]

In [ ]:
# Train Word2Vec model
word2vec_model=Word2Vec(sentences=X_train_tokenized,vector_size=300,window=5,min_count=1,workers=4,sg=1)# sg=1 for Skip-Gram model

In [ ]:
def vectorize_comments(tokenized_comments,word2vec_model):
  vectorized_comments=[]
  for tokens in tokenized_comments:
    vectors=[word2vec_model.wv[token]for token in tokens if token in word2vec_model.wv]
    if len(vectors)>0:
      vectorized_comments.append(np.mean(vectors,axis=0))

    else:
      vectorized_comments.append(np.zeros(word2vec_model.vector_size))
  return np.array(vectorized_comments)

In [ ]:
X_train_word2vec=vectorize_comments(X_train_tokenized,word2vec_model)
X_test_word2vec=vectorize_comments(X_test_tokenized,word2vec_model)

In [ ]:
X_train_word2vec.shape

(29434, 300)

In [ ]:
best_model=LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric='multi_logloss',
    is_unbalance=True,
    class_weight='balanced',
    learning_rate=0.09,
    n_estimators=360,
    max_depth=32,
    reg_alpha=3.053178589412354e-07,
    reg_lambda=7.198246850506435e-08
)

In [ ]:
# Train the LightGBM model on Word2Vec embeddings
best_model.fit(X_train_word2vec, y_train_cleaned)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.138842 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 29434, number of used features: 300
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


LGBMClassifier(class_weight='balanced', is_unbalance=True, learning_rate=0.09,
               max_depth=32, metric='multi_logloss', n_estimators=360,
               num_class=3, objective='multiclass',
               reg_alpha=3.053178589412354e-07,
               reg_lambda=7.198246850506435e-08)

In [ ]:
# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
print(classification_report(y_test_cleaned, y_pred))

              precision    recall  f1-score   support

          -1       0.48      0.49      0.48      1671
           0       0.73      0.72      0.73      2587
           1       0.68      0.69      0.69      3101

    accuracy                           0.65      7359
   macro avg       0.63      0.63      0.63      7359
weighted avg       0.65      0.65      0.65      7359



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# If using tokenized input, set tokenizer & lowercase=False to skip re-tokenizing
tfidf = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
tfidf.fit(X_train_tokenized)

# Build word-to-IDF dictionary
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))



def vectorize_comments_weighted(tokenized_comments, word2vec_model, word2tfidf):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = []
        weights = []
        for token in tokens:
            if token in word2vec_model.wv and token in word2tfidf:
                vectors.append(word2vec_model.wv[token])
                weights.append(word2tfidf[token])
        if vectors:
            vectors = np.array(vectors)
            weights = np.array(weights)
            weighted_vec = np.average(vectors, axis=0, weights=weights)
            vectorized_comments.append(weighted_vec)
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))
    return np.array(vectorized_comments)



X_train_word2vec_wt = vectorize_comments_weighted(X_train_tokenized, word2vec_model, word2tfidf)
X_test_word2vec_wt  = vectorize_comments_weighted(X_test_tokenized, word2vec_model, word2tfidf)




/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

In [ ]:
import xgboost as xgb

# Create DMatrix
dtrain = xgb.DMatrix(X_train_word2vec_wt, label=y_train_cleaned)
dvalid = xgb.DMatrix(X_test_word2vec_wt,  label=y_test_cleaned)

# Set params
params = {
    'objective': 'multi:softprob',
    'num_class': len(np.unique(y_train_cleaned)),
    'eval_metric': 'mlogloss',
    'learning_rate': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'seed': 42
}

# Train with early stopping
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(
    params,
    dtrain,
    num_boost_round=300,
    evals=watchlist,
    early_stopping_rounds=20,
    verbose_eval=True
)

# Predict
y_pred_prob = model.predict(dvalid)
y_pred = np.argmax(y_pred_prob, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test_cleaned, y_pred))


[0]	train-mlogloss:1.08301	eval-mlogloss:1.08484
[1]	train-mlogloss:1.06850	eval-mlogloss:1.07224
[2]	train-mlogloss:1.05508	eval-mlogloss:1.06049
[3]	train-mlogloss:1.04224	eval-mlogloss:1.04920
[4]	train-mlogloss:1.03039	eval-mlogloss:1.03873
[5]	train-mlogloss:1.01896	eval-mlogloss:1.02886
[6]	train-mlogloss:1.00813	eval-mlogloss:1.01949
[7]	train-mlogloss:0.99799	eval-mlogloss:1.01083
[8]	train-mlogloss:0.98847	eval-mlogloss:1.00280
[9]	train-mlogloss:0.97928	eval-mlogloss:0.99500
[10]	train-mlogloss:0.97053	eval-mlogloss:0.98779
[11]	train-mlogloss:0.96213	eval-mlogloss:0.98079
[12]	train-mlogloss:0.95428	eval-mlogloss:0.97428
[13]	train-mlogloss:0.94658	eval-mlogloss:0.96807
[14]	train-mlogloss:0.93932	eval-mlogloss:0.96223
[15]	train-mlogloss:0.93215	eval-mlogloss:0.95635
[16]	train-mlogloss:0.92549	eval-mlogloss:0.95087
[17]	train-mlogloss:0.91914	eval-mlogloss:0.94580
[18]	train-mlogloss:0.91306	eval-mlogloss:0.94109
[19]	train-mlogloss:0.90710	eval-mlogloss:0.93636
[20]	train

In [ ]:
best_model=LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric='multi_logloss',
    is_unbalance=True,
    class_weight='balanced',
    learning_rate=0.09,
    n_estimators=360,
    max_depth=32,
    reg_alpha=3.053178589412354e-07,
    reg_lambda=7.198246850506435e-08
)

In [ ]:
# Train the LightGBM model on Word2Vec embeddings
best_model.fit(X_train_word2vec_wt, y_train_cleaned)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.154955 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 29434, number of used features: 300
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


LGBMClassifier(class_weight='balanced', is_unbalance=True, learning_rate=0.09,
               max_depth=32, metric='multi_logloss', n_estimators=360,
               num_class=3, objective='multiclass',
               reg_alpha=3.053178589412354e-07,
               reg_lambda=7.198246850506435e-08)

In [ ]:
# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec_wt)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
print(classification_report(y_test_cleaned, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.70      0.71      2587
           1       0.66      0.68      0.67      3101
           2       0.45      0.45      0.45      1671

    accuracy                           0.64      7359
   macro avg       0.61      0.61      0.61      7359
weighted avg       0.64      0.64      0.64      7359

